In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model as lm
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf

In [4]:
rogue_data = pd.read_csv('Rogue-142-Fall2021.csv')
rogue_data

,MonthNumeric,MonthFactor,Year,RogueSales,Unemployment,RogueQueries,CPIAll,CPIEnergy
0,1,January,2008,5435,5.0,18,212.174,226.775
1,2,February,2008,5223,4.9,11,212.687,229.731
2,3,March,2008,6873,5.1,18,213.448,233.349
3,4,April,2008,5814,5.0,17,213.942,234.778
4,5,May,2008,7467,5.4,17,215.208,243.924
...,...,...,...,...,...,...,...,...
157,2,February,2021,27750,6.2,95,263.161,218.946
158,3,March,2021,31219,6.0,100,264.793,229.886
159,4,April,2021,32270,6.1,100,266.832,229.658
160,5,May,2021,31029,5.8,96,268.551,229.630


## 1a)

In [5]:
rogue_train = rogue_data[rogue_data['Year'] <= 2015][['Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]
##rogue_val = rogue_data[rogue_data['Year'] <= 2015][['RogueSales']]
rogue_test_A = rogue_data[(rogue_data['Year'] > 2015) & (rogue_data['Year'] <= 2019)][['MonthFactor','Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]
rogue_test_B = rogue_data[rogue_data['Year'] > 2019][['MonthFactor','Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]

display(rogue_train.head(), rogue_test_A.head(), rogue_test_B.head())

,Unemployment,RogueQueries,CPIEnergy,CPIAll,RogueSales
0,5.0,18,226.775,212.174,5435
1,4.9,11,229.731,212.687,5223
2,5.1,18,233.349,213.448,6873
3,5.0,17,234.778,213.942,5814
4,5.4,17,243.924,215.208,7467


,MonthFactor,Unemployment,RogueQueries,CPIEnergy,CPIAll,RogueSales
96,January,4.8,50,185.235,237.652,19762
97,February,4.9,52,176.401,237.336,21561
98,March,5.0,48,182.234,238.080,27713
99,April,5.1,46,186.713,238.992,23173
100,May,4.8,51,188.596,239.557,27428


,MonthFactor,Unemployment,RogueQueries,CPIEnergy,CPIAll,RogueSales
144,January,3.5,78,219.117,258.687,19396
145,February,3.5,80,213.963,258.824,25111
146,March,4.4,58,202.995,257.989,15210
147,April,14.8,45,183.788,256.192,8505
148,May,13.3,67,179.610,255.942,18900


In [6]:
train, val = train_test_split(rogue_train, test_size = 0.15, random_state = 42)
display(len(train), len(val))

81

15

In [7]:
ols_1 = smf.ols(formula = 'RogueSales ~ Unemployment + RogueQueries + CPIEnergy + CPIAll', data = train)
model_1 = ols_1.fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:             RogueSales   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     69.51
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           1.25e-24
Time:                        18:33:31   Log-Likelihood:                -761.86
No. Observations:                  81   AIC:                             1534.
Df Residuals:                      76   BIC:                             1546.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -6.895e+04    1.7e+04     -4.049   

In [8]:
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

cols = ['Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll']
VIF(train, cols)

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Unemployment    2.031353
RogueQueries    6.805939
CPIEnergy       1.649399
CPIAll          5.615907
dtype: float64

In [9]:
ols_2 = smf.ols(formula = 'RogueSales ~ Unemployment + CPIEnergy + CPIAll', data = train)
model_2 = ols_2.fit()
print(model_2.summary())

cols = ['Unemployment', 'CPIEnergy', 'CPIAll']
VIF(train, cols)

                            OLS Regression Results                            
Dep. Variable:             RogueSales   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     83.95
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           3.30e-24
Time:                        18:33:31   Log-Likelihood:                -765.38
No. Observations:                  81   AIC:                             1539.
Df Residuals:                      77   BIC:                             1548.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -1.015e+05   1.21e+04     -8.390   

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Unemployment    1.421546
CPIEnergy       1.125091
CPIAll          1.569258
dtype: float64

In [23]:
def OSR2(model, df_train, df_test, dependent_var):   
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)    
    return 1 - SSE/SST

In [11]:
OSR2(model_2, train, val, 'RogueSales')

0.8448826212090333

In [12]:
OSR2(model_2, train, rogue_test_A, 'RogueSales')

0.8872523114510952

In [13]:
OSR2(model_2, train, rogue_test_B, 'RogueSales')

-0.2535698276625966

## 1b)

In [14]:
rogue_train_month = rogue_data[rogue_data['Year'] <= 2015][['MonthFactor','Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]
train_month, val_month = train_test_split(rogue_train_month, test_size = 0.15, random_state = 42)


In [22]:
ols_3 = smf.ols(formula = 'RogueSales ~ MonthFactor + Unemployment + RogueQueries + CPIEnergy + CPIAll', data = train_month)
model_3 = ols_3.fit()
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:             RogueSales   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     23.50
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           1.40e-20
Time:                        18:35:43   Log-Likelihood:                -748.84
No. Observations:                  81   AIC:                             1530.
Df Residuals:                      65   BIC:                             1568.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [16]:
OSR2(model_3, train_month, val_month, 'RogueSales')

0.8678036276015989

In [17]:
OSR2(model_3, train_month, rogue_test_A, 'RogueSales')

0.90763329508117

In [18]:
OSR2(model_3, train_month, rogue_test_B, 'RogueSales')

-0.7553925283349923

## 1c)

In [24]:
rogue_train_comb = rogue_data[rogue_data['Year'] <= 2015][['MonthFactor', 'Unemployment', 'CPIEnergy', 'CPIAll','RogueSales']]
train_comb, val_comb = train_test_split(rogue_train_month, test_size = 0.15, random_state = 42)


In [25]:
ols_4 = smf.ols(formula = 'RogueSales ~ MonthFactor + Unemployment + CPIEnergy + CPIAll', data = train_comb)
model_4 = ols_4.fit()
print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:             RogueSales   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     23.80
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           1.83e-20
Time:                        18:59:41   Log-Likelihood:                -751.28
No. Observations:                  81   AIC:                             1533.
Df Residuals:                      66   BIC:                             1568.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [26]:
OSR2(model_4, train_comb, val_comb, 'RogueSales')

0.8943409970710554

In [27]:
OSR2(model_4, train_comb, rogue_test_A, 'RogueSales')

0.9015844060375029

In [28]:
OSR2(model_4, train_comb, rogue_test_B, 'RogueSales')

-0.439865503143672

## 1d)

In [22]:
inv_sales_ratio = pd.read_csv('inventory_sales.csv')
inv_sales_ratio

,MonthNumeric,MonthFactor,Year,InventoryToSales
0,1,January,2009,4.64
1,2,February,2009,4.57
2,3,March,2009,4.09
3,4,April,2009,4.01
4,5,May,2009,3.58
...,...,...,...,...
143,12,December,2020,1.88
144,1,January,2021,1.76
145,2,February,2021,1.84
146,3,March,2021,1.35


In [23]:
modified_rogue = rogue_data[rogue_data['Year'] > 2008].reset_index()
modified_rogue = modified_rogue.iloc[0:148,:]
modified_rogue['InventoryToSalesRatio'] = inv_sales_ratio['InventoryToSales']
modified_rogue

,index,MonthNumeric,MonthFactor,Year,RogueSales,Unemployment,RogueQueries,CPIAll,CPIEnergy,InventoryToSalesRatio
0,12,1,January,2009,5802,7.8,15,211.933,178.661,4.64
1,13,2,February,2009,5515,8.3,13,212.705,183.721,4.57
2,14,3,March,2009,6982,8.7,14,212.495,178.415,4.09
3,15,4,April,2009,4758,9.0,14,212.709,177.552,4.01
4,16,5,May,2009,8287,9.4,11,213.022,179.830,3.58
...,...,...,...,...,...,...,...,...,...,...
143,155,12,Decemeber,2020,23338,6.7,91,261.560,203.716,1.88
144,156,1,January,2021,27750,6.3,90,262.231,210.822,1.76
145,157,2,February,2021,27750,6.2,95,263.161,218.946,1.84
146,158,3,March,2021,31219,6.0,100,264.793,229.886,1.35


In [24]:
mod_rogue_train = modified_rogue[modified_rogue['Year'] <= 2015][['MonthFactor', 'InventoryToSalesRatio','Unemployment', 'CPIEnergy', 'CPIAll','RogueSales']]
train_mod, val_mod = train_test_split(mod_rogue_train, test_size = 0.15, random_state = 42)

In [25]:
mod_rogue_test_A = modified_rogue[(modified_rogue['Year'] > 2015) & (modified_rogue['Year'] <= 2019)][['MonthFactor','InventoryToSalesRatio','Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]
mod_rogue_test_B = modified_rogue[modified_rogue['Year'] > 2019][['MonthFactor','InventoryToSalesRatio','Unemployment', 'RogueQueries', 'CPIEnergy', 'CPIAll','RogueSales']]


In [26]:
train_mod

,MonthFactor,InventoryToSalesRatio,Unemployment,CPIEnergy,CPIAll,RogueSales
81,October,2.50,5.0,195.616,237.733,24939
55,August,2.59,7.2,244.833,233.456,17273
54,July,2.46,7.3,242.986,232.900,16992
33,October,2.04,8.8,246.655,226.750,10034
76,May,2.60,5.6,208.973,237.001,25901
...,...,...,...,...,...,...
20,Septeber,2.66,9.5,209.832,218.275,8319
60,January,2.92,6.6,250.340,235.288,13831
71,Decemeber,2.52,5.6,218.536,236.252,14879
14,March,2.37,9.9,209.326,217.353,8395


In [27]:
ols_5 = smf.ols(formula = 'RogueSales ~ MonthFactor + InventoryToSalesRatio + Unemployment + CPIEnergy + CPIAll', data = train_mod)
model_5 = ols_5.fit()
print(model_5.summary())

                            OLS Regression Results                            
Dep. Variable:             RogueSales   R-squared:                       0.858
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     22.24
Date:                Tue, 21 Sep 2021   Prob (F-statistic):           4.73e-18
Time:                        23:04:49   Log-Likelihood:                -652.00
No. Observations:                  71   AIC:                             1336.
Df Residuals:                      55   BIC:                             1372.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [28]:
OSR2(model_5, train_mod, val_mod, 'RogueSales')

0.7246876934105222

In [29]:
OSR2(model_5, train_mod, mod_rogue_test_A, 'RogueSales')

0.8790521605120256

In [30]:
OSR2(model_5, train_mod, mod_rogue_test_B, 'RogueSales')

-0.006927597365599558